In [ ]:
### Setup ###
os.environ['PROJ_LIB'] = '/Applications/anaconda3/share/proj/'
filepath = "./static/psclkls_20200711/"
username = '42'
### Setup End ###

In [ ]:
import json, os, requests, plotly, uuid 
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from bs4 import BeautifulSoup
from user_agents import parse
from datetime import datetime
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import networkx as nx  
import matplotlib.pyplot as plt  


In [ ]:
def get_ip_location(ipadress):
    url = 'http://ipinfo.io/' + ipadress
    r = requests.get(url)
    data = json.loads(r.text)
    y = data["loc"].split(',')[0]
    x = data["loc"].split(',')[1]
    return data['city'], y , x

def get_user_agent(inputstring): 
    useragent = parse(inputstring)
    new_ua = str(useragent.os.family) + " " + str(useragent.os.version)
    return new_ua

def _convert_to_degress(value):
    """Helper function to convert the GPS coordinates stored in the EXIF to degress in float format"""
    d0 = value[0][0]
    d1 = value[0][1]
    d = float(d0) / float(d1)

    m0 = value[1][0]
    m1 = value[1][1]
    m = float(m0) / float(m1)

    s0 = value[2][0]
    s1 = value[2][1]
    s = float(s0) / float(s1)

    return d + (m / 60.0) + (s / 3600.0)

In [ ]:
#Mobile Devices JSON
with open(filepath + 'devices.json') as json_file:
    data = json.load(json_file)
df_devices_mobile = pd.DataFrame.from_dict(data["devices"])
df_devices_camera = pd.DataFrame.from_dict(data["camera"])

#Anonymze Device ID
for index, row in df_devices_mobile.iterrows():
    row["device_id"] = row["device_id"][:8] + (10*"*")
df_devices_mobile

#Devices JSON
with open(filepath + 'account_history.json') as json_file:
    data = json.load(json_file)
df_devices = pd.DataFrame.from_dict(data["login_history"])
df_devices.drop(columns = ["device_id", "language_code"], inplace=True)

#Get Location by IP Adress
df_devices["c_Location"] = ""
df_devices["c_Location_N"] = ""
df_devices["c_Location_E"] = ""
for index, row in df_devices.iterrows():
    row["c_Location"], row["c_Location_N"], row["c_Location_E"] = get_ip_location(row["ip_address"])
    
#Get Hour of Activity
df_devices["c_hour"] = ""
for index, row in df_devices.iterrows():
    row["c_hour"] = datetime.strptime(row.timestamp, "%Y-%m-%dT%H:%M:%S%z").hour

#Parse User Agent
df_devices["c_OS"] = ""
for index, row in df_devices.iterrows():
     row["c_OS"] = get_user_agent(row["user_agent"])

#Anonymze IP Adress
for index, row in df_devices.iterrows():
    row["ip_address"] = row["ip_address"][:8] + (10*"*")

#Merge the Dataframes
df_devices = df_devices.merge(df_devices_mobile, on = "user_agent", how='left')
df_devices.drop(columns= ["last_seen"], inplace = True)
df_devices

In [ ]:
#Output Devices 
print('Number of cameras in phone:', len(df_devices_camera))
print('Number of used phones:', len(df_devices_mobile))

In [ ]:
#At which time I registrered a new device
fig = go.Figure(data=[go.Histogram(x=df_devices["c_hour"], nbinsx=8)])
fig.update_layout(
    title_text='New device registration by hour', # title of plot
    xaxis_title_text='Hour', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [ ]:
df_city = pd.DataFrame()
df_city["Count"] = df_devices["c_Location_N"].value_counts()
df_city["Lat"] = df_devices["c_Location_N"].value_counts().index
df_city["Lon"] = df_devices["c_Location_E"].value_counts().index
df_city.reset_index(inplace=True, drop=True)

#Extract Exif Data
img_list = os.listdir(filepath + "posts")
for i in img_list:
    try:
        img = Image.open(filepath + "posts/" + i)
        exif = {
            TAGS[k]: v
            for k, v in img.getexif().items()
            if k in TAGS
        }
        if "GPSInfo" in exif:
            coordinate = exif["GPSInfo"]
            data = {
                "Count" : 1,
                "Lat" : _convert_to_degress(coordinate[2]),
                "Lon" : _convert_to_degress(coordinate[4]), 
            }
            df_city = df_city.append(data, ignore_index = True)

            print(lat, lon)
    except:
        pass

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

lat = df_city["Lat"].values
lon = df_city["Lon"].values
lats = [float(l) for l in lat]
lons = [float(l) for l in lon]

fig = plt.gcf()
fig.set_size_inches(20, 12.5)

m = Basemap(projection='gnom', resolution='h',
            width=1E6, height=1E6, 
            lat_0=51, lon_0=9.5,)

m.shadedrelief()
m.drawcountries(color='gray')
m.drawstates(color='gray')
m.drawmapboundary(fill_color="#DDEEFF")
m.fillcontinents(color="#a4a4a4", zorder=1)

x, y = m(lons, lats)
plt.scatter(x, y, 130, marker='o', c=df_city["Count"], cmap='Reds', zorder=10, alpha=0.7) 

plt.show()

In [ ]:
#Media (Posts)
with open(filepath + 'media.json') as json_file:
    data = json.load(json_file)
    
photos = pd.DataFrame(data["photos"])
stories = pd.DataFrame(data["stories"])
direct = pd.DataFrame(data["direct"])

#At which time I posted a picture
fig = go.Figure()
fig.add_trace(go.Histogram(x=stories.taken_at.unique(), nbinsx=64, name='Stories'))
fig.add_trace(go.Histogram(x=photos.taken_at.unique(), nbinsx=64, name='Posts'))
fig.update_layout(
    title_text='Posted', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Total', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    barmode='overlay'
)
fig.show()

In [ ]:
##Interactions with others

#Phone Book
with open(filepath + 'uploaded_contacts.json') as json_file:
    data = json.load(json_file)
df_contacts = pd.DataFrame(data)
df_contacts
print("Updated Phone Book", len(df_contacts.imported_time.unique()), "Times")
print(len(df_contacts.contact.unique()), "phone numbers shared")

#Pseudoanonymze Contacts
for index, row in df_contacts.iterrows():
    row["contact"] = row["contact"][:4] + (5*"*")
    row["last_name"] = row["last_name"][:2] + (5*"*")
df_contacts

In [ ]:
#User
with open(filepath + 'connections.json') as json_file:
    data = json.load(json_file)
df_users = pd.DataFrame(data)
df_friends = df_users[df_users.following.notnull() & df_users.followers.notnull()]

#Pseudoanonymze Contacts
df_users["Account"] = df_users.index
df_users.reset_index(inplace=True, drop=True)
for index, row in df_users.iterrows():
    row["Account"] = row["Account"][:4] + (5*"*")
df_users

In [ ]:
#Filtered Users
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_users[df_users.blocked_users.notnull()].blocked_users.unique(), nbinsx=64, name='Block', cumulative_enabled=True))
fig.update_layout(
    title_text='Filtered Users Cummulative', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Total', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    barmode='overlay'
)
fig.show()

In [ ]:
#Friends
#Pseudoanonymze Contacts
df_friends["Account"] = df_friends.index
df_friends.reset_index(inplace=True, drop=True)
for index, row in df_friends.iterrows():
    row["Account"] = uuid.uuid4() 
df_friends.drop(columns=["blocked_users", "hide_stories_from", "follow_requests_sent", "dismissed_suggested_users"], inplace=True)

#Delta between Follower and Following
for index, row in df_friends.iterrows():
    row["followers"] = datetime.strptime(row.followers, "%Y-%m-%dT%H:%M:%S%z")
    row["following"] = datetime.strptime(row.following, "%Y-%m-%dT%H:%M:%S%z")
df_friends["delta"] = df_friends.followers-df_friends.following
df_friends

#Invalid Dates. Reasons? 
#Probably last interaction of them with my profile

In [ ]:
#Finding Friends over Time (Wrong because Issue described above)
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_friends.following.unique(), nbinsx=64, name='Following', cumulative_enabled=True, opacity=0.5))
fig.add_trace(go.Histogram(x=df_friends.followers.unique(), nbinsx=32, name='Last Interaction', cumulative_enabled=True, opacity=0.5))
fig.update_layout(
    title_text='Last Interaction with Friends', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Total', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1, # gap between bars of the same location coordinates
    barmode='overlay',
)
fig.show()

In [ ]:
#Searches
with open(filepath + 'searches.json') as json_file:
    data = json.load(json_file)
    
df_search = pd.DataFrame(data)

#Saved
with open(filepath + 'saved.json') as json_file:
    data = json.load(json_file)
    
df_saved = pd.DataFrame(data["saved_media"])

#Stories seen
with open(filepath + 'stories_activities.json') as json_file:
    data = json.load(json_file)
df_polls = pd.DataFrame(data["polls"])
df_quiz = pd.DataFrame(data["quizzes"])
df_interaction = df_polls.append(df_quiz)
df_interaction = df_interaction.rename(columns={0 : "timestamp", 1 : "author"})
df_interaction["Activity"] = "Story_interaction"

#Seen Content
with open(filepath + 'seen_content.json') as json_file:
    data = json.load(json_file)
df_ads_clicked = pd.DataFrame(data["ads_clicked"])

df_profiles_visited = pd.DataFrame(data["chaining_seen"])
df_profiles_visited = df_profiles_visited.rename(columns={"username" : "author"})
df_profiles_visited["Activity"] = "Profile_visit"

df_search = df_search.rename(columns={"search_click" : "author", "time" : "timestamp"})
df_search.drop(columns = "type", inplace = True)
df_search["Activity"] = "Profile_search"

df_ads_seen = pd.DataFrame(data["ads_seen"])
df_ads_seen["Activity"] = "Ad_seen"

df_posts_seen = pd.DataFrame(data["posts_seen"])
df_posts_seen["Activity"] = "Post_seen"

df_videos_watched = pd.DataFrame(data["posts_seen"])
df_videos_watched["Activity"] = "Video_watched"

df_saved = df_saved.rename(columns={0 : "timestamp", 1 : "author"})
df_saved["Activity"] = "Post_saved"



with open(filepath + 'likes.json') as json_file:
    data = json.load(json_file)
df_likes = pd.DataFrame(data["media_likes"])
df_likes = df_likes.rename(columns={0 : "timestamp", 1 : "author"})
df_likes["Activity"] = "Post_liked"

df_comment_likes = pd.DataFrame(data["comment_likes"])
df_comment_likes = df_comment_likes.rename(columns={0 : "timestamp", 1 : "author"})
df_comment_likes["Activity"] = "Comment_liked"

with open(filepath + 'comments.json') as json_file:
    data = json.load(json_file)
df_comments = pd.DataFrame(data["media_comments"])
df_comments = df_comments.rename(columns={0 : "timestamp", 1:"message", 2 : "author"})
df_comments.drop(columns= "message", inplace=True)
df_comments["Activity"] = "Comment"

df_interactions = df_profiles_visited.append([df_profiles_visited, df_ads_seen, df_posts_seen, df_videos_watched, df_saved, df_search, df_interaction, df_likes, df_comment_likes, df_comments])


df_interactions["c_hour"] = ""
df_interactions["c_date"] = ""
for index, row in df_interactions.iterrows():
    row["c_hour"] = datetime.strptime(row.timestamp, "%Y-%m-%dT%H:%M:%S%z").hour
    row["c_date"] = datetime.strptime(row.timestamp, "%Y-%m-%dT%H:%M:%S%z").date()
df_interactions

In [ ]:
#Pseudoanonymze Contacts (Just for Export, Not for Processing!)
for index, row in df_interactions.iterrows():
    row["author"] = str(row["author"])[:4] + (5*"*")
df_interactions

In [ ]:
#At which time I'm active
fig = go.Figure(data=[go.Histogram(x=df_interactions["c_hour"], nbinsx=24)])
fig.update_layout(
    title_text='Activity by hour', # title of plot
    xaxis_title_text='Hour', # xaxis label
    yaxis_title_text='Total', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
fig.show()

In [ ]:
#At which time I posted a picture
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Profile_visit"]["c_hour"], nbinsx=24, name='Profile_visit', opacity=0.7))
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Post_liked"]["c_hour"], nbinsx=24, name='Post_liked', opacity=0.7))
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Post_saved"]["c_hour"], nbinsx=24, name='Post_saved', opacity=0.7))
fig.update_layout(
    title_text='Activities by Hour', # title of plot
    xaxis_title_text='Hour', # xaxis label
    yaxis_title_text='Total', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.0, # gap between bars of the same location coordinates
    barmode='group'
)
fig.show()

In [ ]:
#Comments, Likes, Posts, Seen, Saved, Messages over time

#At which time I posted a picture
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Profile_visit"]["c_date"], nbinsx=24, name='Profile_visit', opacity=0.7))
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Post_liked"]["c_date"], nbinsx=24, name='Post_liked', opacity=0.7))
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Post_saved"]["c_date"], nbinsx=24, name='Post_saved', opacity=0.7))
fig.add_trace(go.Histogram(x=df_interactions[df_interactions.Activity == "Comment"]["c_date"], nbinsx=24, name='Comment', opacity=0.7))

fig.update_layout(
    title_text='Activities by Date', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Total', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.0, # gap between bars of the same location coordinates
    barmode='group'
)
fig.show()

In [ ]:
#05.2018 - 05.2020
df_interactions['c_date'] = pd.to_datetime(df_interactions['c_date'])  
df_filtered = df_interactions[(df_interactions.c_date >= "2018-05-01")& (df_interactions.c_date <= "2020-05-01")]

df_filtered.groupby("author").filter(lambda x: len(x) > 20).author.unique()

In [ ]:
#Weight User Interactions for Network Plot
weights = {
    "Profile_visit" : 2,
    "Ad_seen" : 0 ,
    "Post_seen" : 0,
    "Video_watched" : 0,
    "Post_saved" : 5 ,
    "Profile_search" : 1,
    "Story_interaction" : 10,
    "Post_liked" : 5,
    "Comment_liked" : 15,
    "Comment" : 20, 
}

df_filtered['Weight'] = df_filtered['Activity'].map(lambda x: weights[x])

df_filtered.head()

In [ ]:
with open(filepath + 'messages.json') as json_file:
    data = json.load(json_file)
  
df_in_contact = pd.DataFrame(columns=['author', 'Msg_Count'])
for i in range(0, len(data)):
    data[i]['participants'].remove(username)
    participant = ",".join(str(x) for x in data[i]['participants'])
    Msg_Count = len(data[i]['conversation'])
    df_in_contact = df_in_contact.append({'author' : participant, 'Msg_Count' : Msg_Count}, ignore_index = True)
    
df_in_contact = df_in_contact.merge(df_filtered, on = 'author')
df_in_contact

In [ ]:
#User I'm in contact with
r = df_in_contact.groupby("author").filter(lambda x: len(x) > 3).author.unique()

edges = []
i = 0 #Initial Node
m = 1 #Number of nodes
weight=[] 

dic_results = {}

for rr in r:  
    m = m + 1
    weight = df_in_contact[df_in_contact.author == rr].Weight.sum()
    msg_count = df_in_contact[df_in_contact.author == rr].Msg_Count.unique()[0]
    weight = weight + round(msg_count / 5, 0)
    edges.append((i, m, { 'myweight':  weight}))
    dic_results[rr] = (weight, m)
        
g = nx.Graph() 
g.add_edges_from(edges)
pos = nx.spring_layout(g, weight='myweight')

fig, axes = plt.subplots(1)
nx.draw_networkx(g ,pos, with_labels = True, node_size = 800, font_size = 16, ax=axes)

plt.show()

In [ ]:
dic_results